##                                                    CCC - NLP to SQL generator

In [3]:
pip install --upgrade -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install arize-phoenix

In [5]:
!pip install "arize-phoenix[evals]" "openai>=1" gcsfs nest-asyncio "llama-index>=0.10.3" "llama-index-core" "openinference-instrumentation-llama-index>=1.0.0" "llama-index-callbacks-arize-phoenix>=0.1.2" "llama-index-readers-wikipedia" "sqlalchemy" wikipedia

In [6]:
#px.__version__
#!pip install pyxlsb
!pip install pandas


In [7]:
import os
from getpass import getpass

import openai
import pandas as pd
import phoenix as px
from llama_index.core import Document, Settings, set_global_handler
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.query_engine import NLSQLTableQueryEngine, RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.tools import QueryEngineTool
from llama_index.core.utilities.sql_wrapper import SQLDatabase
from llama_index.llms.openai import OpenAI
from sqlalchemy import (
    create_engine,
    text,
)

pd.set_option("display.max_colwidth", 1000)

c:\Users\agaur8\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Importing all the libraries

In [8]:
from dotenv import load_dotenv
import openai
import os
import langchain.llms as llms
from langchain.chains import create_sql_query_chain
import pandas as pd
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain.prompts import PromptTemplate
import openpyxl
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [9]:
working_dir = "G:\My Drive\MSBA\Semester4\CCC\LLM-powered-OLAP-CCC-project-main"
os.chdir(working_dir)

print(f"The current working directory: {os.getcwd()}")

The current working directory: g:\My Drive\MSBA\Semester4\CCC\LLM-powered-OLAP-CCC-project-main


In [10]:
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Enable Phoenix tracing within LlamaIndex by setting arize_phoenix as the global handler. This will mount Phoenix's OpenInferenceTraceCallback as the global handler. Phoenix uses OpenInference traces - an open-source standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [11]:
set_global_handler("arize_phoenix")

In [12]:
#Configure Your OpenAI API Key
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

### Importing and loading the csv file to SQLite Database

In [13]:
# Load CSV file
csv_file = 'dataset_flattened.csv'
df = pd.read_csv(csv_file)  

# Save to SQLite DB
db_file = 'dataset.db'  
conn = sqlite3.connect(db_file)
table_name = 'models'
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

print(f"Data from '{csv_file}' has been successfully saved to '{db_file}' in the table '{table_name}'.")


Data from 'dataset_flattened.csv' has been successfully saved to 'dataset.db' in the table 'models'.


In [14]:
# SQLite database connection
db = SQLDatabase.from_uri("sqlite:///dataset.db")

# List all the available table names in the database
print("Table Names:", db.get_usable_table_names())

# Execute the query
db.run("SELECT count(*) from models")


Table Names: ['models']


'[(4797,)]'

### Developing a chain with sample example

In [15]:
# GPT connection
llm = ChatOpenAI(model="gpt-4", temperature=0.2)

#First Chain to take the user input and convert it to a SQL query
write_query = create_sql_query_chain(llm, db)
response = write_query.invoke({"question": "Which production model delivers the top performance for company A?"})
response 

'SELECT "Model Name", "Model Version", "Performance Metrics" FROM models WHERE "Company Name" = \'company A\' ORDER BY "Performance Metrics" DESC LIMIT 1'

In [16]:
#Second chain to excute the SQL queries
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query
chain.invoke({"question": "Which production model delivers the top performance for company A?"})


''

###  Fine tuning the model with basic prompt engineering for our use case

In [17]:
# Prompt to generate SQL queries
SQl_prompt = PromptTemplate.from_template(
    """You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run, to answer the input question.
Remember, your query should aim to answer the question with a single SQL statement and limit the results appropriately. You can order the results to return the most informative data in the database.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
Never query for all columns from a table. Only query the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Be careful not to query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use the date('now') function to get the current date if the question involves "today".

Only use the following tables:

{table_info}.

Question: {input}
Provide up to {top_k} SQL variations."""
)

# Prompt to answer the questions
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
    
Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

In [18]:
# Initialize Chain 1: Generate SQL Query
generate_sql_chain = create_sql_query_chain(prompt=SQl_prompt, llm=llm, db=db)

# Initialize Chain 2: Execute SQL Query and generate structured answer
execute_sql_chain = answer_prompt | llm | StrOutputParser()


def execute_combined_chain(question):
    # Step 1: Generate SQL Query
    sql_query = generate_sql_chain.invoke({"question": question, "top_k": 1})
    
    # Step 2: Execute the SQL Query to get the result
    sql_result = execute_query(sql_query)  # Ensure this returns the result of executing the SQL query
    
    # Step 3: Pass the necessary inputs to the final chain and format the output to include both SQL query and result
    final_response = execute_sql_chain.invoke({"question": question, "query": sql_query, "result": sql_result})
    
    # Format the final answer to include both the SQL query and its result
    final_answer = f"SQL Query: {sql_query}\n Answer: {final_response}"
    return final_answer


In [19]:
# Example invocation
question = "Which production model delivers the top performance for company A?"
final_answer = execute_combined_chain(question)
print(final_answer)

SQL Query: SQL Query:

SELECT "Model Name", "Model Version", "Performance Metrics"
FROM models
WHERE "Company Name" = 'Company A'
ORDER BY "Performance Metrics" DESC
LIMIT 1;
 Answer: The query has an error and did not return a result. Therefore, the top performance model for Company A cannot be determined.


Loading dataframe for ground truth

In [20]:
import pandas as pd

# Read the .xlsx file
df = pd.read_excel('ground_truth.xlsx')

# Display the DataFrame
print(df)

                                                             Question  \
0                          How many records are there in models table   
1                           How many models do we have in production?   
2                Which production model possesses the highest volume?   
3                 Which production model exhibits the highest volume?   
4  Which production model delivers the top performance for company A?   

                                                                                                                                                     SQLQuery  \
0                                                                                                                                 SELECT COUNT(*) FROM models   
1                                                                                                           'SELECT COUNT(DISTINCT "Model Name") FROM models'   
2                                                                             

In [21]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [22]:
%pip install faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [23]:
import numpy as np
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

# Assuming you have already loaded your DataFrame named 'df'
text_data = df['Question'].tolist()

# Train Doc2Vec model
documents = [TaggedDocument(words=str(d).split(), tags=[str(i)]) for i, d in enumerate(text_data)]
model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4, epochs=100)

# Convert documents to vectors
vectors = [model.infer_vector(doc.words) for doc in documents]

# Convert vectors to numpy array
vectors_array = np.array(vectors)

# Use faiss to create an index
import faiss

# Instantiate an index
index = faiss.IndexFlatL2(vectors_array.shape[1])  # L2 distance is commonly used for cosine similarity

# Add vectors to the index
index.add(vectors_array)

# Now you have your DataFrame data converted into a vector index using faiss

Build LlamaIndex Application

In [ ]:
pip install openai

In [25]:
#from openai import OpenAI, Temperature

# Create an instance of the OpenAI class
#openai_instance = OpenAI(model="gpt-4")

#Settings.llm = OpenAI(temperature=0.2, model="gpt-4")

In [26]:

vector_query_engines = [index.as_query_engine() for index in vector_indices]

vector_tools = []
for query_engine in vector_query_engines:
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        description="You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run, to answer the input question. Remember, your query should aim to answer the question with a single SQL statement and limit the results appropriately. You can order the results to return the most informative data in the database. You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again. Never query for all columns from a table. Only query the columns that are needed to answer the question. Wrap each column name in double quotes to denote them as delimited identifiers. Be careful not to query for columns that do not exist. Also, pay attention to which column is in which table.Pay attention to use the date('now') function to get the current date if the question involves 'today",)

NameError: name 'vector_indices' is not defined

Make Queries and Use Phoenix to view Spans

In [ ]:
response = query_engine.query("What is the most expensive digital camera?")
print(str(response))

In [ ]:
response = query_engine.query("Tell me about the history of digital camera sensors.")
print(str(response))

### Further Fine tunning the model

In [ ]:
db = SQLDatabase.from_uri("sqlite:///dataset.db")
print(db.dialect)
print(db.get_usable_table_names())

# Define the SQL query for the view
view_query = """
CREATE VIEW model_metrics_view AS
SELECT
    MODEL_ID,
    CASE 
        WHEN "Model_Type" IN ('Multi-Class', 'Classification') 
            THEN json_extract("Performance_Metrics", '$[0]')
    END AS recall,
    CASE 
        WHEN "Model_Type" IN ('Multi-Class', 'Classification') 
            THEN json_extract("Performance_Metrics", '$[1]')
    END AS precision,
    CASE 
        WHEN "Model_Type" IN ('Multi-Class', 'Classification') 
            THEN json_extract("Performance_Metrics", '$[2]')
    END AS accuracy,
    CASE
        WHEN "Model_type" IN ("Regression")
             THEN Performance_Metrics
    END AS MAE
FROM
    models;
"""

# Execute the query to create the view
db.run(view_query)


sqlite
['models']


OperationalError: (sqlite3.OperationalError) view model_metrics_view already exists
[SQL: 
CREATE VIEW model_metrics_view AS
SELECT
    MODEL_ID,
    CASE 
        WHEN "Model_Type" IN ('Multi-Class', 'Classification') 
            THEN json_extract("Performance_Metrics", '$[0]')
    END AS recall,
    CASE 
        WHEN "Model_Type" IN ('Multi-Class', 'Classification') 
            THEN json_extract("Performance_Metrics", '$[1]')
    END AS precision,
    CASE 
        WHEN "Model_Type" IN ('Multi-Class', 'Classification') 
            THEN json_extract("Performance_Metrics", '$[2]')
    END AS accuracy,
    CASE
        WHEN "Model_type" IN ("Regression")
             THEN Performance_Metrics
    END AS MAE
FROM
    models;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
db.run('SELECT * FROM model_metrics_view')

OperationalError: (sqlite3.OperationalError) no such column: MODEL_ID
[SQL: SELECT * FROM model_metrics_view]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# Updated SQL Prompt
SQl_prompt_updated = PromptTemplate.from_template(
    """You are a SQLite expert and Machine Learning Engineer. Given an input question, create a syntactically correct SQLite query to run, to answer the input question.

    **Important:** 
    * For questions about detailed performance metrics (accuracy, MAE, recall, precision), **YOU MUST** JOIN the 'models' table with the 'model_metrics_view' (which contains structured performance data) on the MODEL_ID column and **USE ONLY** columns in 'model_metrics_view' to get accuracy, recall, MAE and precsion and also add a WHERE condition based on the model_type .
    * **YOU MUST check if the model type first based on that as a ML engineer analyse which metric is suitable**
       * Regression models: Use MAE from model_metrics_view
       * Classification models: Use accuracy, precision, recall from model_metrics_view.
    * If the question might yield multiple results, ensure your query returns ALL relevant entries.  Only limit output if the user explicitly asks for a 'best' result.
    * Remember, your query should aim to answer the question with a single SQL statement and limit the results appropriately. You can order the results to return the most informative data in the database.
    * **Check Data Availability:** Before providing a final response, ensure that the data referred to in the query (Model_Name, date, company) likely exists in the database. If potential issues are detected, indicate the problem and suggest alternatives for the user instead of executing the query.
    **Before Execution:** Double-check your query for errors. Is the JOIN with 'model_metrics_view' used when appropriate?  Does the query structure seem likely to return the correct number of results? If you identify any potential issues, rewrite the query and rerun the corrected query. 

**Important:** Pay close attention to relationships between tables and views.  JOIN tables using appropriate conditions.  

    **Schema Summary:**
    * models table: MODEL_ID, Model_Name, Model_Type, ... 
    * model_metrics_view: MODEL_ID, recall, precision, accuracy, MAE

    **Example:**  
    Question: Which models have recall greater than 0.9?
    SELECT Model_Name 
         FROM models 
         JOIN model_metrics_view ON models.MODEL_ID = model_metrics_view.MODEL_ID
         WHERE Model_type IN ('Multi-Class', 'Classification') and accuracy > 0.9;
    
    Question: What is the accuracy of Model 10?
     SELECT accuracy 
         FROM models 
         JOIN model_metrics_view ON models.MODEL_ID = model_metrics_view.MODEL_ID
         WHERE lower(Model_Name) = 'model 10' and Model_type IN ('Multi-Class', 'Classification');
    Answer: Model 10 is a Regression model type so it accuracy is not the right metric to calculate performace of the model

    Question: What is the volume of Model A in CCC company ?
    SELECT Daily_Volume 
         FROM models 
         WHERE lower(Model_Name) = 'model a' and lower(Company_Name) == 'ccc';
    Answer: It seems there is no model named 'Model A' and no company called CCC in the database.  Can you please try with a different model and company name?
    
    Only use the following tables: {table_info}.
    Question: {input}.
    Generate up to {top_k} SQL queries to answer the question.
    """
)

# Prompt to answer the questions
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question in conversational tone.
      If the SQL result is empty, provide a helpful message indicating that no matching data was found.  

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)


In [ ]:
# Initialize Chain 1: Generate SQL Query
generate_sql_chain = create_sql_query_chain(prompt=SQl_prompt_updated, llm=llm, db=db)

# Initialize Chain 2: Execute SQL Query and generate structured answer
execute_sql_chain = answer_prompt | llm | StrOutputParser()


def execute_combined_chain(question):
    # Step 1: Generate SQL Query
    sql_query = generate_sql_chain.invoke({"question": question, "top_k": 1})
    
    # Step 2: Execute the SQL Query to get the result
    sql_result = execute_query(sql_query)  # Ensure this returns the result of executing the SQL query
    
    # Step 3: Pass the necessary inputs to the final chain and format the output to include both SQL query and result
    final_response = execute_sql_chain.invoke({"question": question, "query": sql_query, "result": sql_result})
    
    # Format the final answer to include both the SQL query and its result
    final_answer = f"SQL Query: {sql_query}\n Answer: {final_response}"
    return final_answer


In [ ]:
# Example invocation
question = "During the timeframe between dates 2022 and 2023, what represents the optimal version of Model 1 in production with regards to model performance?"
final_answer = execute_combined_chain(question)
print(final_answer)

SQL Query: SELECT Model_Version, Model_Type, recall, precision, accuracy, MAE 
FROM models 
JOIN model_metrics_view ON models.MODEL_ID = model_metrics_view.MODEL_ID
WHERE lower(Model_Name) = 'model 1' 
AND Date BETWEEN '2022-01-01' AND '2023-12-31'
ORDER BY CASE 
    WHEN Model_Type IN ('Multi-Class', 'Classification') THEN accuracy 
    WHEN Model_Type = 'Regression' THEN MAE 
    END DESC 
LIMIT 1;
 Answer: I'm sorry, but I couldn't find any data on the optimal version of Model 1 in production between 2022 and 2023. It's possible that the model wasn't in production during that timeframe or there's no recorded data for that period.


In [ ]:
"Which model in production has the best performance for Nationwide?"
"Accuracy of model 10"
"Which model would you sugest for nationwide"
"which model version is better in each model" ## error
"give company wise best performing model"